<a href="https://colab.research.google.com/github/WajeehaAslam/ML-lab/blob/main/genetic_algo_ml_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random

def random_chromosome(length=5):
    return ''.join(random.choice('01') for _ in range(length))

def fitness(chromosome):
    return int(chromosome, 2) ** 2

def selection(population):
    return random.choices(
        population, weights=[fitness(ch) for ch in population], k=2)

def crossover(p1, p2):
    point = random.randint(1, len(p1) - 1)
    return p1[:point] + p2[point:], p2[:point] + p1[point:]

def mutate(chromosome, rate=0.1):
    return ''.join(
        bit if random.random() > rate else random.choice('01') for bit in chromosome)

def genetic_algorithm(pop_size=6, generations=20):
    population = [random_chromosome() for _ in range(pop_size)]

    for gen in range(generations):
        population = sorted(population, key=fitness, reverse=True)
        print(f"Gen {gen}: Best = {population[0]}, Fitness = {fitness(population[0])}")

        next_gen = population[:2]  # Elitism
        while len(next_gen) < pop_size:
            p1, p2 = selection(population)
            c1, c2 = crossover(p1, p2)
            next_gen += [mutate(c1), mutate(c2)]

        population = next_gen[:pop_size]

    return max(population, key=fitness)

best = genetic_algorithm()
print("Best Solution:", best, "→", int(best, 2), "^2 =", fitness(best))

Gen 0: Best = 11010, Fitness = 676
Gen 1: Best = 11011, Fitness = 729
Gen 2: Best = 11111, Fitness = 961
Gen 3: Best = 11111, Fitness = 961
Gen 4: Best = 11111, Fitness = 961
Gen 5: Best = 11111, Fitness = 961
Gen 6: Best = 11111, Fitness = 961
Gen 7: Best = 11111, Fitness = 961
Gen 8: Best = 11111, Fitness = 961
Gen 9: Best = 11111, Fitness = 961
Gen 10: Best = 11111, Fitness = 961
Gen 11: Best = 11111, Fitness = 961
Gen 12: Best = 11111, Fitness = 961
Gen 13: Best = 11111, Fitness = 961
Gen 14: Best = 11111, Fitness = 961
Gen 15: Best = 11111, Fitness = 961
Gen 16: Best = 11111, Fitness = 961
Gen 17: Best = 11111, Fitness = 961
Gen 18: Best = 11111, Fitness = 961
Gen 19: Best = 11111, Fitness = 961
Best Solution: 11111 → 31 ^2 = 961


In [2]:
import random
import numpy as np

# Adjacency matrix based on the graph (n1 to n11 mapped as 0 to 10)
# 999 indicates no direct edge (but path can be completed via other nodes)
graph = [
    [0, 4, 3, 999, 999, 999, 999, 999, 999, 999, 999],  # n1 (0)
    [4, 0, 1, 999, 999, 4, 999, 999, 999, 999, 999],    # n2 (1)
    [3, 1, 0, 1, 7, 999, 3, 999, 999, 999, 999],        # n3 (2)
    [999, 999, 1, 0, 2, 999, 1, 999, 999, 999, 999],    # n4 (3)
    [999, 999, 7, 2, 0, 2, 1, 999, 999, 4, 999],        # n5 (4)
    [999, 4, 999, 999, 2, 0, 5, 6, 999, 999, 999],      # n6 (5)
    [999, 999, 3, 1, 1, 5, 0, 3, 1, 999, 999],          # n7 (6)
    [999, 999, 999, 999, 999, 6, 3, 0, 1, 999, 4],      # n8 (7)
    [999, 999, 999, 999, 999, 999, 1, 1, 0, 2, 1],      # n9 (8)
    [999, 999, 999, 999, 4, 999, 999, 999, 2, 0, 2],    # n10 (9)
    [999, 999, 999, 999, 999, 999, 999, 4, 1, 2, 0]     # n11 (10)
]

# Parameters
POPULATION_SIZE = 50
GENERATIONS = 100
MUTATION_RATE = 0.1

# Calculate fitness (total distance of the route)
def calculate_fitness(route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += graph[route[i]][route[i + 1]]
    # Add return to start
    total_distance += graph[route[-1]][route[0]]
    return total_distance

# Create initial population
def create_population(size, num_cities):
    population = []
    for _ in range(size):
        route = list(range(num_cities))
        random.shuffle(route)
        population.append(route)
    return population

# Tournament selection
def select_parents(population, k=3):
    tournament = random.sample(population, k)
    return min(tournament, key=calculate_fitness)

# Ordered crossover
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    # Copy the segment from parent1
    child[start:end] = parent1[start:end]
    # Fill remaining positions with parent2's genes in order
    remaining = [gene for gene in parent2 if gene not in child[start:end]]
    pos = 0
    for i in range(size):
        if child[i] is None:
            child[i] = remaining[pos]
            pos += 1
    return child

# Swap mutation
def mutate(route):
    if random.random() < MUTATION_RATE:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]
    return route

# Genetic Algorithm
def genetic_algorithm():
    num_cities = len(graph)
    population = create_population(POPULATION_SIZE, num_cities)

    for generation in range(GENERATIONS):
        # Evaluate population
        population = sorted(population, key=calculate_fitness)

        # Create new population
        new_population = population[:2]  # Elitism: keep top 2

        while len(new_population) < POPULATION_SIZE:
            parent1 = select_parents(population)
            parent2 = select_parents(population)
            child = crossover(parent1, parent2)
            child = mutate(child)
            new_population.append(child)

        population = new_population

    # Return the best route
    best_route = min(population, key=calculate_fitness)
    best_distance = calculate_fitness(best_route)
    return best_route, best_distance

# Run the algorithm
best_route, best_distance = genetic_algorithm()

# Print results (for reference, not part of artifact output)
print(f"Best Route (0-based indices): {best_route}")
print(f"Best Distance: {best_distance}")


Best Route (0-based indices): [4, 9, 10, 8, 7, 5, 1, 0, 2, 3, 6]
Best Distance: 28


In [3]:
from numpy.random import randint, rand
import numpy as np

# Objective function
def objective(x):
    return x[0]**2.0 + x[1]**2.0

# Decode bitstring to real values
def decode(bounds, n_bits, bitstring):
    decoded = []
    largest = 2**n_bits - 1
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i + 1) * n_bits
        substring = bitstring[start:end]
        # convert bitstring to string and then to integer
        chars = ''.join([str(s) for s in substring])
        integer = int(chars, 2)
        # scale to bounds
        value = bounds[i][0] + (integer / largest) * (bounds[i][1] - bounds[i][0])
        decoded.append(value)
    return decoded

# Tournament selection
def selection(pop, scores, k=3):
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# Crossover two parents to create two children
def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if rand() < r_cross:
        pt = randint(1, len(p1)-1)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# Mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if rand() < r_mut:
            bitstring[i] = 1 - bitstring[i]

# Genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))

    for gen in range(n_iter):
        decoded = [decode(bounds, n_bits, p) for p in pop]
        scores = [objective(d) for d in decoded]
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen, decoded[i], scores[i]))

        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            for c in crossover(p1, p2, r_cross):
                mutation(c, r_mut)
                children.append(c)
        pop = children
    return [best, best_eval]

# Problem definition
bounds = [[-5.0, 5.0], [-5.0, 5.0]]
n_iter = 100
n_bits = 16
n_pop = 100
r_cross = 0.9
r_mut = 1.0 / (float(n_bits) * len(bounds))

# Run the algorithm
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
decoded = decode(bounds, n_bits, best)
print('Done!')
print('f(%s) = %f' % (decoded, score))



>0, new best f([-2.086289768825818, -2.6093690394445717]) = 11.161412
>0, new best f([3.079957274738689, 0.9870298313878081]) = 10.460365
>0, new best f([-2.0934615091172653, -2.143663691157397]) = 8.977875
>0, new best f([-0.09605554283970452, 0.6589608606088344]) = 0.443456
>1, new best f([-0.09590295262073756, 0.6382085908293273]) = 0.416508
>1, new best f([-0.09605554283970452, -0.5013351644159609]) = 0.260564
>1, new best f([0.34218356603341693, 0.35637445639734455]) = 0.244092
>2, new best f([0.34218356603341693, 0.35591668574044366]) = 0.243766
>3, new best f([0.34889753566796333, 0.1902037079423211]) = 0.157907
>3, new best f([-0.09605554283970452, 0.014419775692378067]) = 0.009435
>4, new best f([-0.052109559777218095, 0.05226214999618506]) = 0.005447
>5, new best f([-0.052109559777218095, 0.042954146639200275]) = 0.004560
>5, new best f([-0.04234378576333242, 0.05226214999618506]) = 0.004524
>6, new best f([-0.04234378576333242, 0.042954146639200275]) = 0.003638
>6, new best 